In [93]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [94]:
# Configurar el navegador (Chrome en este caso)
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")  # Maximiza la ventana

# Inicializar el WebDriver (Asegúrate de tener el driver adecuado instalado)
driver = webdriver.Chrome(options=options)

# URL de la página de consulta
url = "https://www.rues.org.co/?old=true"

# Abrir la página
driver.get(url)

# Esperar a que la página cargue completamente
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "txtNIT")))

print("Página cargada correctamente")


Página cargada correctamente


In [95]:
# Recargar la página para evitar que aparezca el modal
driver.refresh()

# Esperar a que la página cargue completamente
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "txtNIT"))
)

print("✅ Página recargada. Modal evitado.")



✅ Página recargada. Modal evitado.


In [96]:
# Número de matrícula a buscar (Ejemplo: "0000224776")
nit_empresa = "900474867"

# Encontrar el campo de entrada y escribir la matrícula
campo_nit = driver.find_element(By.ID, "txtNIT")
campo_nit.clear()  # Limpiar el campo antes de escribir
campo_nit.send_keys(nit_empresa)  # Ingresar la matrícula

# Hacer clic en el botón de búsqueda
boton_buscar = driver.find_element(By.ID, "btnConsultaNIT")
boton_buscar.click()

# Esperar a que los resultados carguen
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "rmTable2")))

print("Consulta realizada correctamente")

#hacer un delay de 3 segundos
time.sleep(3)


Consulta realizada correctamente


In [97]:
# Esperar que la tabla de resultados esté visible
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "rmTable2")))

# Buscar la fila de la tabla
fila_resultado = driver.find_element(By.XPATH, "//table[@id='rmTable2']/tbody/tr")

# Extraer los datos de la fila
nit = fila_resultado.find_elements(By.TAG_NAME, "td")[0].text.strip()
razon_social = fila_resultado.find_elements(By.TAG_NAME, "td")[1].text.strip()
sigla = fila_resultado.find_elements(By.TAG_NAME, "td")[2].text.strip()
municipio_departamento = fila_resultado.find_elements(By.TAG_NAME, "td")[3].text.strip()
categoria = fila_resultado.find_elements(By.TAG_NAME, "td")[4].text.strip()

# Guardar la información en un diccionario
datos_empresa = {
    "NIT": nit,
    "Razón Social": razon_social,
    "Sigla": sigla,
    "Municipio/Dpto": municipio_departamento,
    "Categoría": categoria
}

# Imprimir los datos obtenidos
print("Datos extraídos de la tabla de resultados:")
print(datos_empresa)

 



Datos extraídos de la tabla de resultados:
{'NIT': 'NIT 900474867 - 7', 'Razón Social': 'ENTER TELECOMUNICACIONES BANDA ANCHA SAS', 'Sigla': '', 'Municipio/Dpto': 'SAN AGUSTIN / HUILA', 'Categoría': 'SOCIEDAD ó PERSONA JURIDICA PRINCIPAL ó ESAL'}


In [98]:
from selenium.webdriver.common.action_chains import ActionChains

# Encontrar la celda del NIT que contiene el botón
celda_nit = fila_resultado.find_element(By.XPATH, ".//td[1]")  # Primera columna (NIT)

# Realizar un scroll hacia el elemento para asegurarnos de que es visible
driver.execute_script("arguments[0].scrollIntoView();", celda_nit)

# Intentar hacer clic en la celda (esto podría abrir el detalle)
try:
    celda_nit.click()
except Exception:
    print("🔹 Celda del NIT no interactuable directamente, probando con JavaScript.")

    # Si el click normal no funciona, usar JavaScript para forzar la interacción
    driver.execute_script("arguments[0].click();", celda_nit)

# Esperar a que la página de detalles cargue completamente
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "card-title")))

print("✅ Redirigido correctamente a la página de detalles.")



🔹 Celda del NIT no interactuable directamente, probando con JavaScript.
✅ Redirigido correctamente a la página de detalles.


In [99]:
# Obtener el enlace ya presente en la página
enlace_detalle = driver.find_element(By.XPATH, "//a[contains(@href, 'ConsultarDetalleRM')]")

# Realizar un scroll hacia el elemento para asegurarnos de que es visible
driver.execute_script("arguments[0].scrollIntoView();", enlace_detalle)

# Hacer clic en el enlace para abrir la página de detalles
try:
    enlace_detalle.click()
except Exception:
    print("🔹 Enlace no interactuable directamente, probando con JavaScript.")
    driver.execute_script("arguments[0].click();", enlace_detalle)

# Esperar a que la nueva página cargue completamente
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "card-title")))

print("✅ Redirigido correctamente a la página de detalles.")


🔹 Enlace no interactuable directamente, probando con JavaScript.
✅ Redirigido correctamente a la página de detalles.


In [100]:
#hacer un delay de 3 segundos
time.sleep(3)

# Esperar a que la tabla de información esté disponible
# Esperar a que la tabla de información esté disponible
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "table"))
)

# Buscar la tabla dentro de la sección correspondiente
tabla_info = driver.find_element(By.XPATH, "//div[contains(@class, 'card-text')]/table")

# Extraer las filas de la tabla
filas = tabla_info.find_elements(By.TAG_NAME, "tr")

# Diccionario para almacenar los datos extraídos temporalmente
datos_empresa = {}

# Recorrer las filas de la tabla
for fila in filas:
    columnas = fila.find_elements(By.TAG_NAME, "td")
    if len(columnas) == 2:
        clave = columnas[0].text.strip()
        valor = columnas[1].text.strip()
        datos_empresa[clave] = valor

# Crear variables individuales a partir del diccionario
numero_matricula = datos_empresa.get("Numero de Matricula", "N/A")
ultimo_anio_renovado = datos_empresa.get("Último Año Renovado", "N/A")
fecha_renovacion = datos_empresa.get("Fecha de Renovacion", "N/A")
fecha_matricula = datos_empresa.get("Fecha de Matricula", "N/A")
fecha_vigencia = datos_empresa.get("Fecha de Vigencia", "N/A")
estado_matricula = datos_empresa.get("Estado de la matricula", "N/A")
motivo_cancelacion = datos_empresa.get("Motivo Cancelación", "N/A")
tipo_sociedad = datos_empresa.get("Tipo de Sociedad", "N/A")
tipo_organizacion = datos_empresa.get("Tipo de Organización", "N/A")
categoria_matricula = datos_empresa.get("Categoria de la Matricula", "N/A")
fecha_ultima_actualizacion = datos_empresa.get("Fecha Ultima Actualización", "N/A")
emprendimiento_social = datos_empresa.get("Emprendimiento Social", "N/A")

# Imprimir las variables creadas
print("✅ Variables creadas correctamente:")
print(f"Numero de Matricula: {numero_matricula}")
print(f"Último Año Renovado: {ultimo_anio_renovado}")
print(f"Fecha de Renovación: {fecha_renovacion}")
print(f"Fecha de Matrícula: {fecha_matricula}")
print(f"Fecha de Vigencia: {fecha_vigencia}")
print(f"Estado de la Matrícula: {estado_matricula}")
print(f"Motivo Cancelación: {motivo_cancelacion}")
print(f"Tipo de Sociedad: {tipo_sociedad}")
print(f"Tipo de Organización: {tipo_organizacion}")
print(f"Categoría de la Matrícula: {categoria_matricula}")
print(f"Fecha Última Actualización: {fecha_ultima_actualizacion}")
print(f"Emprendimiento Social: {emprendimiento_social}")



✅ Variables creadas correctamente:
Numero de Matricula: 224776
Último Año Renovado: 2025
Fecha de Renovación: 20250103
Fecha de Matrícula: 20111031
Fecha de Vigencia: Indefinida
Estado de la Matrícula: ACTIVA
Motivo Cancelación: NORMAL
Tipo de Sociedad: SOCIEDAD COMERCIAL
Tipo de Organización: SOCIEDADES POR ACCIONES SIMPLIFICADAS SAS
Categoría de la Matrícula: SOCIEDAD ó PERSONA JURIDICA PRINCIPAL ó ESAL
Fecha Última Actualización: 20250103
Emprendimiento Social: N


In [101]:
# Esperar a que la sección de Actividades Económicas esté disponible
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'card border-info')]//ul[@class='cleanlist']"))
)

# Ubicar la lista de actividades económicas
lista_actividades = driver.find_elements(By.XPATH, "//div[contains(@class, 'card border-info')]//ul[@class='cleanlist']/li")

# Lista para almacenar las actividades económicas como diccionarios { "Código": ..., "Descripción": ... }
actividades_economicas = []

# Recorrer cada elemento de la lista y extraer el código y la descripción
for actividad in lista_actividades:
    texto = actividad.text.strip()
    if texto:  # Evitar elementos vacíos
        partes = texto.split(" ", 1)  # Separar por el primer espacio (para código y descripción)
        if len(partes) == 2:
            codigo, descripcion = partes
            actividades_economicas.append({"Código": codigo, "Descripción": descripcion})
        else:
            # Si no hay descripción, solo almacenar el código
            actividades_economicas.append({"Código": partes[0], "Descripción": "No especificada"})

# Mostrar las actividades extraídas
print("✅ Actividades Económicas extraídas correctamente:")
for actividad in actividades_economicas:
    print(f"- Código: {actividad['Código']}, Descripción: {actividad['Descripción']}")





✅ Actividades Económicas extraídas correctamente:
- Código: 6110, Descripción: Actividades de telecomunicaciones alámbricas
- Código: 6120, Descripción: Actividades de telecomunicaciones inalámbricas
- Código: 6190, Descripción: Otras actividades de telecomunicaciones


In [102]:
# Esperar a que el botón "Representantes Legales" esté presente y sea interactuable
boton_representantes = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "Facultades"))
)

# Hacer clic en el botón
boton_representantes.click()

print("✅ Se ha hecho clic en el botón 'Representantes Legales'.")


✅ Se ha hecho clic en el botón 'Representantes Legales'.


In [ ]:
# Esperar a que el contenedor de Facultades esté presente en la página
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "txtFacultades"))
)

# Obtener el contenido del div donde está la información de Representantes Legales y Facultades
contenedor_facultades = driver.find_element(By.ID, "txtFacultades")

# Extraer el texto, eliminando etiquetas HTML y caracteres especiales
informacion_representantes = contenedor_facultades.text.replace("\n", " ").replace("\xa0", " ").strip()

# Mostrar la información extraída
print("✅ Información de Representantes Legales y Facultades extraída correctamente:")
print(informacion_representantes)



In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Esperar a que el botón "Enviar" esté presente y sea interactuable
boton_enviar = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//input[@type='submit' and @value='Enviar']"))
)

# Hacer clic en el botón
boton_enviar.click()

print("✅ Se ha hecho clic en el botón 'Enviar'.")

In [ ]:
#cerrar el navegador
#driver.quit()

In [103]:
# Crear un diccionario con toda la información obtenida
informacion_empresa = {
    "NIT": nit_empresa,
    "Razón Social": razon_social, 
    "Municipio/Departamento": municipio_departamento,  
    "Número de Matrícula": numero_matricula,
    "Último Año Renovado": ultimo_anio_renovado,
    "Fecha de Renovación": fecha_renovacion,
    "Fecha de Matrícula": fecha_matricula,
    "Fecha de Vigencia": fecha_vigencia,
    "Estado de la Matrícula": estado_matricula,
    "Motivo de Cancelación": motivo_cancelacion,
    "Tipo de Sociedad": tipo_sociedad,
    "Tipo de Organización": tipo_organizacion,
    "Categoría de la Matrícula": categoria_matricula,
    "Fecha Última Actualización": fecha_ultima_actualizacion,
    "Emprendimiento Social": emprendimiento_social,
    "Actividades Económicas": actividades_economicas,  # Lista de actividades con código y descripción
    "Representantes": informacion_representantes  # Texto en formato limpio
}

# Imprimir la información de manera organizada
import json
print(json.dumps(informacion_empresa, indent=4, ensure_ascii=False))


{
    "NIT": "900474867",
    "Razón Social": "ENTER TELECOMUNICACIONES BANDA ANCHA SAS",
    "Municipio/Departamento": "SAN AGUSTIN / HUILA",
    "Número de Matrícula": "224776",
    "Último Año Renovado": "2025",
    "Fecha de Renovación": "20250103",
    "Fecha de Matrícula": "20111031",
    "Fecha de Vigencia": "Indefinida",
    "Estado de la Matrícula": "ACTIVA",
    "Motivo de Cancelación": "NORMAL",
    "Tipo de Sociedad": "SOCIEDAD COMERCIAL",
    "Tipo de Organización": "SOCIEDADES POR ACCIONES SIMPLIFICADAS SAS",
    "Categoría de la Matrícula": "SOCIEDAD ó PERSONA JURIDICA PRINCIPAL ó ESAL",
    "Fecha Última Actualización": "20250103",
    "Emprendimiento Social": "N",
    "Actividades Económicas": [
        {
            "Código": "6110",
            "Descripción": "Actividades de telecomunicaciones alámbricas"
        },
        {
            "Código": "6120",
            "Descripción": "Actividades de telecomunicaciones inalámbricas"
        },
        {
            "Cód